## Sections of code copied/adapted from PICASO tutorial:
https://natashabatalha.github.io/picaso/

### The conceptual portion of questions are answered in the provided pdf document.

# Part 1. Reflected Light Spectra of Cool Planets
## 1.
Following the tutorial “Basics of Reflected Light”, plot the albedo spectrum of a
Jupiter-like atmosphere (without any clouds for now) in the wavelength range 0.3 - 1.0
micron. Vary the spectral resolution (R) between 50 and 10,000. What qualitative
changes in the spectrum do you notice when you increase the spectral resolution? What
does that tell you about the nature of the main sources of opacity in this atmosphere?

In [1]:
#Import modules
from picaso import justdoit as jdi
from picaso import justplotit as jpi
import numpy as np

jpi.output_notebook()

Loading BokehJS ...

In [2]:
#Connect to opacity database
opacity = jdi.opannection(wave_range=[0.3,1]) #wavelength range set to 0.3 - 1.0 micron

start_case = jdi.inputs() #Load blank slate

#Customize planet and star properties
start_case.phase_angle(0) #set phase ange in radians
start_case.gravity(gravity=25, gravity_unit=jdi.u.Unit('m/(s**2)'))
start_case.star(opacity, 5000,0,4.0) #opacity db, pysynphot database, temp, metallicity, logg
start_case.atmosphere(filename=jdi.jupiter_pt(), delim_whitespace=True)

In [3]:
#Generate spectrum of Jupiter's reflected light at full phase.
df = start_case.spectrum(opacity)
df.keys()

dict_keys(['wavenumber', 'albedo', 'bond_albedo', 'fpfs_reflected'])

In [4]:
#'R' Adjusts the Spectral Resolution. Vary between 50 and 10,000.
wno, alb, fpfs = df['wavenumber'] , df['albedo'] , df['fpfs_reflected']
wno, alb = jdi.mean_regrid(wno, alb, R=150)

In [5]:
#Plot the spectrum
jpi.show(jpi.spectrum(wno, alb, plot_width=500,x_range=[0.3,1],
                      title='Jupiter-like Atmosphere Spectrum, Spectral Resolution R = 150'))

## 2.
Investigate which molecules (among those commonly found in the atmospheres of cold
giant planets, e.g. water, methane, ammonia, carbon monoxide, carbon dioxide, …) are
the dominant sources of opacity for your model atmosphere, in the wavelength range
specified above?

In [6]:
# Create empty lists to store future values.
w,a,l =[],[],[]

# Iterate through a list of molecules to exclude in the spectra.
for iex in ['H2O','CH4','NH3',['CO','CO2'],None]:
    start_case.atmosphere(filename = jdi.jupiter_pt(),exclude_mol=iex, delim_whitespace=True) # Where the molecules is excluded.
    df_iex = start_case.spectrum(opacity)
    wno, alb, fpfs  = df_iex['wavenumber'] , df_iex['albedo'], df_iex['fpfs_reflected']
    wno, alb = jdi.mean_regrid(wno, alb, R=150)
    # Add the calculated values to the previously defined empty lists.
    w +=[wno]
    a +=[alb]
    # Create the labels for the plot.
    if iex==None:
        leg='Nothing Removed'
    else:
        leg = f'{iex} Removed'
    l+=[leg]
# Plot the spectrum.
jpi.show(jpi.spectrum(w,a,legend=l,title='Jupiter-like Atmosphere Spectrum with Molecules Removed'))

## 3.

g0: Asymmetry factor (0-1), w0: Single scattering albedo (0-1), opd: Tau, p: Pressure level (log10 bars), dp: Cloud thickness (log10 bars)

g0: Asymmetry factor (0-1), w0: Single scattering albedo (0-1), opd: Tau, p: Pressure level (log10 bars), dp: Cloud thickness (log10 bars)

1. g0 doesnt seem to effect much, 
3. opd determines the line noise, opd at 1, less noise, opd at 0, more noise

In [7]:
# Specify the characteristics of the cloud

start_case.clouds( g0=[0.9], w0=[0.8], opd=[0.5], p = [0.0], dp=[1.0])  # Slab cloud from 1.0 bar up to 0.1 bar
df_cld = start_case.spectrum(opacity)
wno_cld, alb_cld, fpfs_cld = df_cld['wavenumber'], df_cld['albedo'], df_cld['fpfs_reflected']
wno_cld, alb_cld = jdi.mean_regrid(wno_cld, alb_cld, R=150)

# Create the figure of the spectrum
fig_basic_cloud = jpi.spectrum(wno_cld,alb_cld, plot_width=500, title='Jupiter-like Cloudy Atmosphere, Control Factors')

In [8]:
# Create list of values with each list possessing one index with an altered value

g0_list = [0.9, 0.1, 0.9, 0.9, 0.9, 0.9]
w0_list = [0.8, 0.8, 0.1, 0.8, 0.8, 0.8]
opd_list = [0.5, 0.5, 0.5, 0.1, 0.5, 0.5]
p_list = [0.0, 0.0, 0.0, 0.0, 0.5, 0.0]
dp_list = [1.0, 1.0, 1.0, 1.0, 1.0, 0.5]

# Create empty lists to store future values
wc,ac =[],[]
# Create labels
labels = ['Simple Clouds', 'g0 = 0.1', 'w0 = 0.1', 'opd = 0.1', 'p = 0.5', 'dp = 0.5']

# Iterate through values in each list to create 6 different cases
for i in range(6):
    start_case.clouds(g0=[g0_list[i]], w0=[w0_list[i]], opd=[opd_list[i]], p=[p_list[i]], dp=[dp_list[i]])
    df_cld = start_case.spectrum(opacity)
    wno_cld, alb_cld, fpfs_cld = df_cld['wavenumber'], df_cld['albedo'], df_cld['fpfs_reflected']
    wno_cld, alb_cld = jdi.mean_regrid(wno_cld, alb_cld, R=150)
    wc +=[wno_cld]
    ac +=[alb_cld]

# Save figure
fig_varying_cloud1 = jpi.spectrum(wc,ac,legend=labels,plot_width=700,title='Jupiter-like Cloudy Atmosphere Spectrum, Varying Factors')

# Create different list of values with each list possessing one index with an altered value

g0_list2 = [0.9, 0.5, 0.9, 0.9, 0.9, 0.9]
w0_list2 = [0.8, 0.8, 0.5, 0.8, 0.8, 0.8]
opd_list2 = [0.5, 0.5, 0.5, 1.0, 0.5, 0.5]
p_list2 = [0.0, 0.0, 0.0, 0.0, 0.1, 0.0]
dp_list2 = [1.0, 1.0, 1.0, 1.0, 1.0, 0.3]

# Create empty lists to store future values
wc2,ac2 =[],[]
# Create labels
labels2 = ['Simple Clouds', 'g0 = 0.5', 'w0 = 0.5', 'opd = 1.0', 'p = 0.1', 'dp = 0.3']

# Iterate through values in each list to create 6 different cases
for i in range(6):
    start_case.clouds(g0=[g0_list2[i]], w0=[w0_list2[i]], opd=[opd_list2[i]], p=[p_list2[i]], dp=[dp_list2[i]])
    df_cld = start_case.spectrum(opacity)
    wno_cld, alb_cld, fpfs_cld = df_cld['wavenumber'], df_cld['albedo'], df_cld['fpfs_reflected']
    wno_cld, alb_cld = jdi.mean_regrid(wno_cld, alb_cld, R=150)
    wc2 +=[wno_cld]
    ac2 +=[alb_cld]

# Save figure
fig_varying_cloud2 = jpi.spectrum(wc2,ac2,legend=labels2,plot_width=700,title='Jupiter-like Cloudy Atmosphere Spectrum, Varying Factors')

# Plot figures
jpi.show(fig_basic_cloud)
jpi.show(jpi.column(fig_varying_cloud1, fig_varying_cloud2))

## 4.
Explanation included in attached written report.

# Part 2. Transmission Spectra of Warm Planets
## 5. 
Following the tutorial “Computing Transmission Spectroscopy”, plot the transmission
spectrum for a hot Jupiter model atmosphere in the wavelength range accessible with
the NIRSpec instrument onboard the James Webb Space Telescope (0.6 - 5 micron).

In [9]:
# Create a new hot Jupiter template
opa_hotJ = jdi.opannection(wave_range=[0.6,5])
hotJ_case = jdi.inputs()
hotJ_case.phase_angle(0)

# Here we are going to have to specify gravity through R and M since we need it in the Flux calc
hotJ_case.gravity(mass=1, mass_unit=jdi.u.Unit('M_jup'),
              radius=1.2, radius_unit=jdi.u.Unit('R_jup'))


# Here we are going to have to specify R as well
hotJ_case.star(opa_hotJ, 4000,0.0122,4.437,radius=0.7, radius_unit = jdi.u.Unit('R_sun') )

# Generate atmosphere
hotJ_case.atmosphere(filename = jdi.HJ_pt(), delim_whitespace=True)

# Reference pressure
hotJ_case.approx(p_reference=10)

In [10]:
# Generate the spectra
df_hotJ = hotJ_case.spectrum(opa_hotJ, full_output=True, calculation='transmission')

wno_hotJ, rprs2_hotJ = df_hotJ['wavenumber'], df_hotJ['transit_depth']
wno_hotJ, rprs2_hotJ = jdi.mean_regrid(wno_hotJ, rprs2_hotJ, R=150)
full_output_hotJ = df_hotJ['full_output']

# plot the spectrum
jpi.show(jpi.spectrum(wno_hotJ,rprs2_hotJ,plot_width=500, title='Spectrum of Hot Jupiter Model Atmosphere in 0.6 - 5 micron'))

# 6.
Investigate which atomic and molecular species contribute significantly to the opacity of
your model atmosphere in the wavelength range of NIRSpec. Common species include
water, carbon monoxide, carbon dioxide, methane, sodium, potassium, etc., but feel free
to explore other species as well. Discuss your findings.

In [11]:
#Default gravity case is mass=1, radius=1.2
test_mass = 1
test_radius = 1.2

hotJ_case.gravity(mass=test_mass, mass_unit=jdi.u.Unit('M_jup'),
              radius=test_radius, radius_unit=jdi.u.Unit('R_jup'))

#atmo
w,f,l =[],[],[]
for iex in ['H2O','CO2','CH4','NA','K',None]:
    hotJ_case.atmosphere(filename = jdi.HJ_pt(),exclude_mol=iex, delim_whitespace=True)
    df_iex = hotJ_case.spectrum(opa_hotJ, full_output=True,calculation='transmission') #note the new last key
    wno, rprs2  = df_iex['wavenumber'] , df_iex['transit_depth']
    wno, rprs2 = jdi.mean_regrid(wno, rprs2, R=150)
    w +=[wno]
    f+=[rprs2]
    if iex==None:
        leg='all'
    else:
        leg = f'No {iex}'
    l+=[leg]
jpi.show(jpi.spectrum(w,f,legend=l, title='Spectrum of Hot Jupiter Model Atmosphere, Various Molecules Removed'))

# 7. 
Vary the surface gravity of your planet. What effect does this have on the resulting
spectrum? Discuss the physical reasons behind this.

In [12]:
# Specify the mass and radius for each case. Cannot specify gravity directly, so will only be increasing mass
mass = [1, 2, 3, 4]
radius = [1.2, 1.2, 1.2, 1.2]

# Labels for each line
label_grav = ['M = 1*Mj', 'M = 2*Mj', 'M = 3*Mj', 'M = 4*Mj']

# Define the atmosphere
hotJ_case.atmosphere(filename = jdi.HJ_pt(), delim_whitespace=True)

# Empty lists for each line
w,f =[],[]
# Iterate through the mass and radius to test the different gravity values
for i in range(4):
    # Define the gravity case
    hotJ_case.gravity(mass=mass[i], mass_unit=jdi.u.Unit('M_jup'), radius=radius[i], radius_unit=jdi.u.Unit('R_jup'))
    df_grav = hotJ_case.spectrum(opa_hotJ, calculation='transmission') #note the new last key
    wno, rprs2  = df_grav['wavenumber'] , df_grav['transit_depth']
    wno, rprs2 = jdi.mean_regrid(wno, rprs2, R=150)
    w +=[wno]
    f +=[rprs2]

# Plot the cases
jpi.show(jpi.spectrum(w,f,legend=label_grav, title='Spectrum of Hot Jupiter Model Atmosphere, Various Gravity Values Tested'))

# 8.
Add a simple model of grey (wavelength independent) clouds to your atmosphere. What
is the effect of clouds on the mean transit depth and the amplitude of spectral features

In [13]:
# Define cloud case, specify characteristics
hotJ_case.clouds(p=[1], dp=[4], opd=[1], g0=[0], w0=[0])
df_gry = hotJ_case.spectrum(opa_hotJ, full_output=True, calculation='transmission')

wno_gry, rprs2_gry = df_gry['wavenumber'], df_gry['transit_depth']
wno_gry, rprs2_gry = jdi.mean_regrid(wno_gry, rprs2_gry, R=150)
full_output_gry = df_gry['full_output']

# Plot the cloud case
jpi.show(jpi.spectrum([wno_gry, wno_gry],[1e6*(rprs2-np.mean(rprs2))
                                         ,(rprs2_gry-np.mean(rprs2_gry))*1e6],
                     legend=['Cloud free','Gray cloud'],
                     plot_width=750, title = 'Spectrum of Hot Jupiter Model Atmosphere, Clouds Added' ))

# Part 3. Thermal Emission Spectra of Warm Planets
## 9.
Following the tutorial “Computing Thermal Fluxes”, plot the thermal emission spectrum
for a hot Jupiter model atmosphere in the NIRSpec wavelength range.

In [14]:
# Define the conditions for this case in NIRSpec wavelength range
opa_hotJ = jdi.opannection(wave_range=[0.6,5])
hotJ_case2 = jdi.inputs()
hotJ_case2.phase_angle(0)


# here we are going to have to specify gravity through R and M since we need it in the Flux calc
hotJ_case2.gravity(mass=1, mass_unit=jdi.u.Unit('M_jup'),
              radius=1.2, radius_unit=jdi.u.Unit('R_jup'))

# here we are going to have to specify R as well
hotJ_case2.star(opa_hotJ, 4000,0.0122,4.437,radius=0.7, radius_unit = jdi.u.Unit('R_sun') )

# Define atmosphere
hotJ_case2.atmosphere(filename = jdi.HJ_pt(), delim_whitespace=True)#

#can uncomment for clouds to explore!
#case1.clouds(filename = jdi.HJ_cld(), delim_whitespace=True)

In [15]:
# Calculate the data for the thermal emission case
df_therm= hotJ_case2.spectrum(opa_hotJ, full_output=True,calculation='thermal') #note the new last key

wno_therm, fpfs_therm, fp_therm = df_therm['wavenumber'] , df_therm['fpfs_thermal'], df_therm['thermal']
wno_bin_therm, fpfs_bin_therm = jdi.mean_regrid(wno_therm, fpfs_therm, R=150)
wno_bin_therm, fp_bin_therm = jdi.mean_regrid(wno_therm, fp_therm, R=150)
full_output_therm = df_therm['full_output']

# Plot the spectrum
jpi.show(jpi.spectrum(wno_bin_therm,fpfs_bin_therm*1e6,plot_width=500,y_axis_type='log', title='Thermal Emission Spectrum For Hot Jupiter Model Atmosphere'))

## 10.
To first order, the emission spectrum looks like a blackbody spectrum with some spectral
features (absorption or emission lines) added. What blackbody temperature (or a
temperature range) provides a reasonably good match to the underlying continuum of
your spectrum? At what pressure level is this temperature reached in your model?

In [16]:
# Plot the computed flux and blackbodies of temperatures at various pressures along the PT profile
jpi.show(jpi.flux_at_top(df_therm, pressures=[1e-1,5e-1,1e-2,1e-4],
                         R=150,  plot_width=500, plot_height=400))

## 11.
Plot the relative abundance (i.e. mixing ratios) of different species as a function of
pressure level in the atmosphere. What is the composition of your atmosphere at the
pressure level you identified in the previous section? Describe how the composition
changes as a function of altitude.

In [17]:
# Plot the mixing ratios and pressure-temperature profile
jpi.show(jpi.row(
    jpi.mixing_ratio(full_output_therm, plot_height=950),
    jpi.pt(full_output_therm, plot_height=950)))